In [4]:
from einops import rearrange

In [3]:
start = torch.rand(size=(4, 1, 128, 128, 128))

In [5]:
embedder = torch.nn.Conv3d(kernel_size=16, stride=16, out_channels=128, in_channels=1)

In [6]:
embedder(start).shape

torch.Size([4, 128, 8, 8, 8])

In [9]:
embedded = embedder(start)
tokens = rearrange(embedded, "b em x y z -> b (x y z) em")

In [10]:
tokens.shape

torch.Size([4, 512, 128])

In [1]:
from src.model.vit import ViT
import torch


In [2]:
model = ViT(embed_dim=128, channels=8, patch_size=16, levels=3)

In [3]:
data = [torch.rand(4, 8, 128, 128, 128), torch.rand(4, 8, 64, 64, 64), torch.rand(4, 8, 32, 32, 32)]
out = model(data)

In [7]:
[print(o.shape) for o in out]
print(2)

torch.Size([4, 8, 128, 128, 128])
torch.Size([4, 8, 64, 64, 64])
torch.Size([4, 8, 32, 32, 32])
2


In [1]:
from src.model.my_model import MyModel
from src.model.vit import DefaultViT
import torch
from src.constants import *
from src.model.baselines import SWINUNETR, UNet
import importlib

In [2]:
device= "cuda"


img = torch.rand(4, 1, 128, 128, 128).to(device)
mask = torch.rand(4, 3, 128, 128, 128).to(device)

tokens = torch.rand(size=(4, 1024, 256)).to(device)


In [3]:
model = MyModel(in_channels=1, out_channels=3, skip_transformer=False, channels=(32, 32, 32, 32, 32),transformer_channels=8, embed_dim=256).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATES[0])

other = UNet.to(device)

In [5]:
%%time
out = model(img)
# loss = LOSS(out, mask)
# loss.backward()
# optimizer.step()

Embed: 0.011003 seconds
ViT: 0.008349 seconds
De-Embed: 0.002454 seconds
CPU times: user 568 ms, sys: 7.26 ms, total: 575 ms
Wall time: 570 ms


In [5]:
%%time


UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


In [ ]:
encoder_layer = torch.nn.TransformerEncoderLayer(d_model=512, nhead=8, dim_feedforward=2048, dropout=0.1,
                                                    activation="gelu", layer_norm_eps=1e-5, batch_first=False, norm_first=False)

encoder_norm = torch.nn.LayerNorm(512, eps=1e-5)
encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=6, norm=encoder_norm).cuda()

In [3]:
example = torch.rand(size=(4, 1024, 512)).cuda()

In [7]:
import os

os.system("nvidia-smi")

Thu Apr 20 17:10:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:3B:00.0 Off |                    0 |
|  0%   42C    P0    79W / 300W |  16707MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A40          On   | 00000000:5E:00.0 Off |                    0 |
|  0%   

0

In [5]:
encoder(example).shape

torch.Size([4, 1024, 512])

In [ ]:
import torch
import time

example = torch.rand(size=(4, 1, 128, 128, 128))
# Synchronize the device to ensure all operations are completed
torch.cuda.synchronize()

# Record the start time
start_time = time.time()

# Perform some operation on the GPU
out = model.vit.embedders

# Synchronize the device again to make sure the operation is completed
torch.cuda.synchronize()

# Record the end time
end_time = time.time()

# Calculate the time taken for the operation
time_elapsed = end_time - start_time
print(f'Time taken for matrix multiplication: {time_elapsed:.6f} seconds')